In [ ]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD
!pwd

In [ ]:
import dataset
import object_detection
from utils import *
import eval

import torch
from torch.utils.data import Dataset, DataLoader

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Using <cuda> device


In [ ]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [ ]:
from torch.backends import cudnn
import random

cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# ====== Args Init ====== #
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 17

# ====== Dataset ====== #
args.val_split = 0.1
args.ch_option = {'num_ch': 3,
                  'img_type': 'visible'}

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)


# ====== Optimizer & Training ====== #
args.optim = 'Adam' 
args.lr = 5e-4
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 100
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [ ]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-47',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-48
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## train

In [ ]:
def train(train_loader, model, criterion, optimizer):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]

        # Forward prop.
        predicted_locs, predicted_scores = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    train_loss = losses.avg
    return train_loss        

## validation

In [ ]:
def validation(val_loader, model, criterion):
    model.eval()

    num_batches = len(val_loader)
    losses = AverageMeter()
    with torch.no_grad():
        for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
            images = images.to(device)
            bboxes = [b.to(device) for b in bboxes]
            category_ids = [l.to(device) for l in category_ids]

            predicted_locs, predicted_scores = model(images)
            loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

            losses.update(loss, images.size(0))

    val_loss = losses.avg
    return val_loss

## checkpoint

In [ ]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [ ]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SSD300(n_classes=args.n_classes,
                                    base=args.base_model,
                                    ch_option=args.ch_option)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.MultiBoxLoss(priors_cxcy=model.priors_cxcy).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [ ]:
workers = 4
train_dataset = dataset.KaistPDDataset(ch_option=args.ch_option)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

In [ ]:
epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5
eval_freq = 10


# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train_loss = train(train_loader=train_loader,
                       model=model,
                       criterion=criterion,
                       optimizer=optimizer)
    run['train/loss'].log(train_loss)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

    if epoch % eval_freq == 0:
        eval.evaluate(model, ch_option=args.ch_option)

# ====== Epoch 1 ====== # 2021-07-29 04:16:50.496102


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 114.654 (114.654)	Data Time 108.856 (108.856)	Loss 12.3052 (12.3052)	
[68/273]	Batch Time 35.360 (10.990)	Data Time 34.774 (10.368)	Loss 3.6018 (4.3226)	
[136/273]	Batch Time 28.672 (10.237)	Data Time 28.117 (9.654)	Loss 3.3188 (3.8604)	
[204/273]	Batch Time 32.947 (9.989)	Data Time 32.390 (9.418)	Loss 3.1972 (3.6780)	
[272/273]	Batch Time 26.526 (9.849)	Data Time 21.549 (9.269)	Loss 3.0043 (3.5470)	
# ====== Epoch 2 ====== # 2021-07-29 05:01:39.383217
[0/273]	Batch Time 7.767 (7.767)	Data Time 7.121 (7.121)	Loss 3.0614 (3.0614)	
[68/273]	Batch Time 6.683 (2.232)	Data Time 6.077 (1.620)	Loss 2.8783 (3.0157)	
[136/273]	Batch Time 0.589 (2.155)	Data Time 0.005 (1.542)	Loss 2.7881 (2.9855)	
[204/273]	Batch Time 0.613 (2.149)	Data Time 0.000 (1.534)	Loss 2.8021 (2.9546)	
[272/273]	Batch Time 0.468 (2.128)	Data Time 0.000 (1.515)	Loss 2.8464 (2.9354)	
# ====== Epoch 3 ====== # 2021-07-29 05:11:20.536166
[0/273]	Batch Time 8.989 (8.989)	Data Time 8.362 (8.362)	Loss 2.8650 

/content/URP_PD/object_detection.py:393: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.27s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 44.33%
Recall: 0.6960859554873369
# ====== Epoch 11 ====== # 2021-07-29 06:52:19.831398


...

[0/273]	Batch Time 10.003 (10.003)	Data Time 9.356 (9.356)	Loss 1.7336 (1.7336)	
[68/273]	Batch Time 2.190 (2.406)	Data Time 1.587 (1.787)	Loss 1.7413 (1.8279)	
[136/273]	Batch Time 0.597 (2.374)	Data Time 0.000 (1.755)	Loss 1.9372 (1.8121)	
[204/273]	Batch Time 0.610 (2.340)	Data Time 0.009 (1.722)	Loss 1.7098 (1.8073)	
[272/273]	Batch Time 0.470 (2.302)	Data Time 0.000 (1.685)	Loss 1.8151 (1.8019)	
# ====== Epoch 72 ====== # 2021-07-29 17:11:44.184410
[0/273]	Batch Time 9.146 (9.146)	Data Time 8.511 (8.511)	Loss 2.3272 (2.3272)	
[68/273]	Batch Time 6.820 (2.364)	Data Time 6.212 (1.748)	Loss 1.8834 (1.8141)	
[136/273]	Batch Time 3.742 (2.333)	Data Time 3.131 (1.719)	Loss 2.1732 (1.8273)	
[204/273]	Batch Time 3.802 (2.340)	Data Time 3.176 (1.724)	Loss 1.4612 (1.8095)	
[272/273]	Batch Time 1.110 (2.302)	Data Time 0.639 (1.688)	Loss 1.8337 (1.8082)	
# ====== Epoch 73 ====== # 2021-07-29 17:22:12.800328
[0/273]	Batch Time 9.986 (9.986)	Data Time 9.311 (9.311)	Loss 1.5669 (1.5669)	
[68/273

/content/URP_PD/object_detection.py:393: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.30s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 34.98%
Recall: 0.770941438102298
# ====== Epoch 81 ====== # 2021-07-29 18:46:26.035517


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0/273]	Batch Time 9.312 (9.312)	Data Time 8.623 (8.623)	Loss 1.6779 (1.6779)	
[68/273]	Batch Time 0.613 (2.422)	Data Time 0.001 (1.805)	Loss 2.0940 (1.7779)	
[136/273]	Batch Time 0.648 (2.358)	Data Time 0.005 (1.741)	Loss 1.7945 (1.7599)	
[204/273]	Batch Time 0.610 (2.316)	Data Time 0.005 (1.700)	Loss 1.8750 (1.7569)	
[272/273]	Batch Time 0.469 (2.282)	Data Time 0.000 (1.667)	Loss 1.5621 (1.7535)	
# ====== Epoch 82 ====== # 2021-07-29 18:56:49.054846
[0/273]	Batch Time 9.238 (9.238)	Data Time 8.595 (8.595)	Loss 1.8088 (1.8088)	
[68/273]	Batch Time 0.608 (2.376)	Data Time 0.001 (1.758)	Loss 2.2250 (1.7744)	
[136/273]	Batch Time 0.633 (2.340)	Data Time 0.006 (1.724)	Loss 1.4049 (1.7490)	
[204/273]	Batch Time 0.626 (2.324)	Data Time 0.002 (1.708)	Loss 1.4982 (1.7602)	
[272/273]	Batch Time 0.467 (2.277)	Data Time 0.000 (1.663)	Loss 1.7956 (1.7562)	
# ====== Epoch 83 ====== # 2021-07-29 19:07:10.737357
[0/273]	Batch Time 10.766 (10.766)	Data Time 10.170 (10.170)	Loss 1.7957 (1.7957)	
[68/2